In [1]:
import mxnet as mx
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import autograd as ag
from mxnet import ndarray as nd
from Algorithm.CNN import CNN_Model
from Algorithm.MLP import MLP
from Tools import utils
import numpy as np
import copy
import random

In [2]:
ctx = utils.try_all_gpus()
net0 = CNN_Model('LeNet')
net0.initialize(init=mx.init.Xavier(),ctx=ctx)
x = nd.random.uniform(0,255,shape=(1,1,28,28),ctx=ctx[0])
net0(x)
net1 = CNN_Model('LeNet')
net1.initialize(init=mx.init.Xavier(),ctx=ctx)
net1(x)


[[ 28.787878  -13.466257  -10.675999  -10.426249  -14.412338   13.499657
   27.438347  -11.734835   21.004559    2.2865102]]
<NDArray 1x10 @gpu(0)>

In [19]:
def zero_net(net,ctx):
    net.initialize(mx.init.Xavier(),ctx=ctx)
    x = nd.random.uniform(0,255,shape=(1,1,28,28),ctx=ctx[0])
    net(x)
    for layer in net:
        try:
            layer.weight.data()[:] = nd.zeros(shape=layer.weight.data().shape,ctx=ctx[0])[:]
            layer.bias.data()[:] = nd.zeros(shape=layer.bias.data().shape,ctx=ctx[0])[:]
        except:
            pass

def merge(net, net_a, net_b):
    id = 0
    for layer in net:
        try:
            layer.weight.data()[:] = net_a[id].weight.data()[:] + net_b[id].weight.data()[:]
            layer.weight.data()[:] /= 2
        except:
            continue
        id += 1

In [7]:
net = CNN_Model('LeNet')
net.load_parameters("LeNet.params",ctx)

In [11]:
zero_net(net,ctx)

In [15]:
net[0].weight.data()[0]


[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]
<NDArray 1x5x5 @gpu(0)>

In [14]:
net1[0].weight.data()[0]


[[[ 0.1363274  -0.17125472 -0.12498833  0.00307944  0.04279496]
  [-0.12337571 -0.13931     0.1033404   0.12887721  0.13514502]
  [ 0.11380883 -0.03281228  0.02558994 -0.13332824 -0.03437263]
  [-0.17286083 -0.15954955  0.17871104  0.0731134  -0.046924  ]
  [-0.01720445 -0.02959834  0.08223341 -0.16642982  0.13568164]]]
<NDArray 1x5x5 @gpu(0)>

In [17]:
net0[0].weight.data()[0]


[[[ 0.01807702  0.03438295  0.07969065  0.12749125  0.03805615]
  [ 0.13255729  0.0166215   0.12859704 -0.02827276  0.04575911]
  [ 0.05402867 -0.0428167  -0.02311321 -0.07497861  0.14508452]
  [-0.16416161  0.17170732 -0.08419175 -0.04316488 -0.00827123]
  [ 0.10803397  0.11560483  0.0107006  -0.00741501  0.0251988 ]]]
<NDArray 1x5x5 @gpu(0)>

In [20]:
merge(net,net0,net1)

In [21]:
net[0].weight.data()[0]


[[[ 0.07720221 -0.06843589 -0.02264884  0.06528535  0.04042555]
  [ 0.00459079 -0.06134425  0.11596872  0.05030222  0.09045207]
  [ 0.08391875 -0.03781449  0.00123837 -0.10415342  0.05535594]
  [-0.16851121  0.00607888  0.04725965  0.01497426 -0.02759761]
  [ 0.04541476  0.04300325  0.04646701 -0.08692241  0.08044022]]]
<NDArray 1x5x5 @gpu(0)>

In [22]:
net.load_parameters('current_model.params',ctx)

In [24]:
le = CNN_Model("LeNet")
le.load_parameters('LeNet.params',ctx)

In [25]:
net[0].weight.data()[0]


[[[ 0.0412934   0.13729984  0.24973604  0.30977833  0.15521261]
  [ 0.20059519  0.13257036  0.274703    0.12228141  0.13972278]
  [ 0.07319901 -0.0200493   0.00734211 -0.03435873  0.18222198]
  [-0.21676877  0.11536153 -0.1263506  -0.08242974 -0.04972405]
  [ 0.01500483  0.01938555 -0.06805743 -0.09051672 -0.07760767]]]
<NDArray 1x5x5 @gpu(0)>

In [26]:
le[0].weight.data()[0]


[[[ 0.01807702  0.03438295  0.07969065  0.12749125  0.03805615]
  [ 0.13255729  0.0166215   0.12859704 -0.02827276  0.04575911]
  [ 0.05402867 -0.0428167  -0.02311321 -0.07497861  0.14508452]
  [-0.16416161  0.17170732 -0.08419175 -0.04316488 -0.00827123]
  [ 0.10803397  0.11560483  0.0107006  -0.00741501  0.0251988 ]]]
<NDArray 1x5x5 @gpu(0)>

In [28]:

for layer in net:
    try:
        print(layer.weight.data()[0])
    except:
        continue


[[[ 0.0412934   0.13729984  0.24973604  0.30977833  0.15521261]
  [ 0.20059519  0.13257036  0.274703    0.12228141  0.13972278]
  [ 0.07319901 -0.0200493   0.00734211 -0.03435873  0.18222198]
  [-0.21676877  0.11536153 -0.1263506  -0.08242974 -0.04972405]
  [ 0.01500483  0.01938555 -0.06805743 -0.09051672 -0.07760767]]]
<NDArray 1x5x5 @gpu(0)>

[[[ 0.02922854  0.01713338 -0.07450107  0.00780702  0.0928883 ]
  [ 0.05402304  0.00456396 -0.08232322 -0.01782653 -0.0055147 ]
  [-0.04918273 -0.06552303  0.05728555  0.04949059 -0.0091599 ]
  [-0.0592107   0.01429538 -0.07620043 -0.10052153 -0.03673574]
  [ 0.02457325 -0.07318053  0.02341603 -0.05800517  0.02442671]]

 [[-0.02371168  0.09269594  0.08410008  0.03798102 -0.01045511]
  [-0.02934784  0.02361819 -0.0131536   0.08404787  0.0412838 ]
  [-0.0837076  -0.0918659   0.09813991  0.03483643  0.0319899 ]
  [ 0.03564508 -0.06874475 -0.06049911 -0.02963104 -0.07751477]
  [ 0.05236663 -0.03855576  0.02252509 -0.02846989 -0.03654644]]

 [[ 0.01

In [30]:
client = CNN_Model("LeNet")
server = CNN_Model("LeNet")
client.load_parameters("client.params")
server.load_parameters("server.params")

In [33]:
cx = client[6].weight.data()
sx = server[6].weight.data()

In [36]:
client[6]

Dense(84 -> 10, linear)

In [38]:
client[6].weight.data()[0]


[ 3.22201401e-02 -2.81384829e-02  3.51844341e-01 -1.54381454e-01
  3.75474319e-02 -5.31216115e-02 -4.80307378e-02  5.96967824e-02
 -8.44299607e-03 -3.14886519e-03 -2.55339980e-01  7.99102150e-03
 -1.13449074e-01  7.12600350e-02  7.04433173e-02 -1.06792741e-01
 -1.06209174e-01 -5.99146374e-02  2.16954970e-04  1.90345317e-01
  3.07289809e-02  2.79137820e-01 -2.26334885e-01 -2.69706219e-01
 -1.07951894e-01 -2.74319500e-01  1.41484305e-01 -1.31204531e-01
  1.12953603e-01  9.98244360e-02  1.86102241e-02 -3.42783540e-01
  1.25624269e-01  2.62751043e-01  1.02170989e-01 -8.18302929e-02
 -3.48010957e-02 -1.40844464e-01  2.01278284e-01 -2.43172660e-01
 -1.80587582e-02 -2.03534961e-01 -2.27170065e-02  2.42795661e-01
  2.79820025e-01 -1.34742498e-01  1.88499644e-01  1.88029464e-02
  7.70080462e-02 -1.88799147e-02 -2.38350838e-01 -1.98407397e-01
  2.11864755e-01 -7.93268532e-03  1.19147897e-01  4.26594587e-03
  2.69022584e-01  1.66525081e-01 -1.99843928e-01  3.10741127e-01
 -2.39677802e-01 -2.0661

In [39]:
server[6].weight.data()[0]


[ 0.00066918  0.0167788   0.23045504 -0.13615929  0.07275698 -0.05312161
 -0.03847538  0.06003293  0.05375955 -0.01269922 -0.24294743 -0.01509194
 -0.10026249  0.10918054  0.08093429 -0.10712627 -0.10607193 -0.05888551
  0.05963215  0.12590331 -0.03599255  0.19122854 -0.18419173 -0.20066963
 -0.10192616 -0.20596987  0.03535265 -0.07374893  0.04591721  0.02618232
  0.0375559  -0.23565511  0.07741103  0.23696223  0.07685643 -0.09044851
 -0.03465365 -0.14084351  0.2003715  -0.18126619 -0.06691982 -0.203501
 -0.03240688  0.24458227  0.19803542 -0.12109761  0.15471712  0.01870716
  0.10302311 -0.02631246 -0.20200182 -0.2023342   0.21196085 -0.07462561
  0.10825425 -0.01553813  0.252063    0.17237511 -0.17713067  0.20446494
 -0.2394014  -0.19839483 -0.23834257  0.00495705 -0.1282373  -0.20103756
  0.18191895 -0.10675901  0.01962098 -0.13554008  0.02669048  0.23630849
  0.17282519 -0.11201352 -0.1899019  -0.1345982  -0.11157653 -0.20638293
  0.04333341  0.03540406  0.23728257 -0.04147099  0.

In [40]:
le[6].weight.data()[0]


[ 0.00066918  0.0167788   0.23045504 -0.13615929  0.07275698 -0.05312161
 -0.03847538  0.06003293  0.05375955 -0.01269922 -0.24294743 -0.01509194
 -0.10026249  0.10918054  0.08093429 -0.10712627 -0.10607193 -0.05888551
  0.05963215  0.12590331 -0.03599255  0.19122854 -0.18419173 -0.20066963
 -0.10192616 -0.20596987  0.03535265 -0.07374893  0.04591721  0.02618232
  0.0375559  -0.23565511  0.07741103  0.23696223  0.07685643 -0.09044851
 -0.03465365 -0.14084351  0.2003715  -0.18126619 -0.06691982 -0.203501
 -0.03240688  0.24458227  0.19803542 -0.12109761  0.15471712  0.01870716
  0.10302311 -0.02631246 -0.20200182 -0.2023342   0.21196085 -0.07462561
  0.10825425 -0.01553813  0.252063    0.17237511 -0.17713067  0.20446494
 -0.2394014  -0.19839483 -0.23834257  0.00495705 -0.1282373  -0.20103756
  0.18191895 -0.10675901  0.01962098 -0.13554008  0.02669048  0.23630849
  0.17282519 -0.11201352 -0.1899019  -0.1345982  -0.11157653 -0.20638293
  0.04333341  0.03540406  0.23728257 -0.04147099  0.

In [41]:
zero_net(le,ctx)
le[6].weight.data()[0]


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
<NDArray 84 @gpu(0)>

In [46]:
id = 0
for layer in server:
    try:
        layer.weight.data()[:] = client[id].weight.data()[:]
        print(id)
    except:
        pass
    id += 1 

0
2
4
5
6


In [47]:
client[6].weight.data()[0]


[ 3.22201401e-02 -2.81384829e-02  3.51844341e-01 -1.54381454e-01
  3.75474319e-02 -5.31216115e-02 -4.80307378e-02  5.96967824e-02
 -8.44299607e-03 -3.14886519e-03 -2.55339980e-01  7.99102150e-03
 -1.13449074e-01  7.12600350e-02  7.04433173e-02 -1.06792741e-01
 -1.06209174e-01 -5.99146374e-02  2.16954970e-04  1.90345317e-01
  3.07289809e-02  2.79137820e-01 -2.26334885e-01 -2.69706219e-01
 -1.07951894e-01 -2.74319500e-01  1.41484305e-01 -1.31204531e-01
  1.12953603e-01  9.98244360e-02  1.86102241e-02 -3.42783540e-01
  1.25624269e-01  2.62751043e-01  1.02170989e-01 -8.18302929e-02
 -3.48010957e-02 -1.40844464e-01  2.01278284e-01 -2.43172660e-01
 -1.80587582e-02 -2.03534961e-01 -2.27170065e-02  2.42795661e-01
  2.79820025e-01 -1.34742498e-01  1.88499644e-01  1.88029464e-02
  7.70080462e-02 -1.88799147e-02 -2.38350838e-01 -1.98407397e-01
  2.11864755e-01 -7.93268532e-03  1.19147897e-01  4.26594587e-03
  2.69022584e-01  1.66525081e-01 -1.99843928e-01  3.10741127e-01
 -2.39677802e-01 -2.0661

In [48]:
server[6].weight.data()[0]


[ 3.22201401e-02 -2.81384829e-02  3.51844341e-01 -1.54381454e-01
  3.75474319e-02 -5.31216115e-02 -4.80307378e-02  5.96967824e-02
 -8.44299607e-03 -3.14886519e-03 -2.55339980e-01  7.99102150e-03
 -1.13449074e-01  7.12600350e-02  7.04433173e-02 -1.06792741e-01
 -1.06209174e-01 -5.99146374e-02  2.16954970e-04  1.90345317e-01
  3.07289809e-02  2.79137820e-01 -2.26334885e-01 -2.69706219e-01
 -1.07951894e-01 -2.74319500e-01  1.41484305e-01 -1.31204531e-01
  1.12953603e-01  9.98244360e-02  1.86102241e-02 -3.42783540e-01
  1.25624269e-01  2.62751043e-01  1.02170989e-01 -8.18302929e-02
 -3.48010957e-02 -1.40844464e-01  2.01278284e-01 -2.43172660e-01
 -1.80587582e-02 -2.03534961e-01 -2.27170065e-02  2.42795661e-01
  2.79820025e-01 -1.34742498e-01  1.88499644e-01  1.88029464e-02
  7.70080462e-02 -1.88799147e-02 -2.38350838e-01 -1.98407397e-01
  2.11864755e-01 -7.93268532e-03  1.19147897e-01  4.26594587e-03
  2.69022584e-01  1.66525081e-01 -1.99843928e-01  3.10741127e-01
 -2.39677802e-01 -2.0661